# 1 - 遍历单个域名

即使你没听说过“维基百科六度分隔理论”，也很可能听过“凯文· 贝肯（Kevin Bacon） 的六度分隔值游戏”。在这两个游戏中，都是把两个不相干的主题（维基百科里是用词条之间的连接，凯文· 贝肯的六度分隔值游戏是用出现在同一部电影中的演员来连接）用一个总数不超过六条的主题连接起来（包括原来的两个主题）。

比如，埃里克· 艾德尔和布兰登· 弗雷泽都出现在电影《骑警杜德雷》里，布兰登· 弗雷泽又和凯文· 贝肯都出现在电影《我呼吸的空气》里。1 因此，根据这两个条件，从埃里克· 艾德尔到凯文·贝肯的链条主题长度只有 3。

我们将在本节创建一个项目来实现“维基百科六度分隔理论”的查找方法。也就是说，我们要实现从埃里克· 艾德尔的词条页面（https://zh.wikipedia.ahau.cf/wiki/Kevin_Bacon） 开始，经过最少的链接点击次数找到凯文· 贝肯的词条页面（https://zh.wikipedia.ahau.cf/wiki/威廉·莫斯里） 。

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib import request

url = "https://zh.wikipedia.ahau.cf/wiki/%E5%87%AF%E6%96%87%C2%B7%E8%B4%9D%E8%82%AF"
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
req = request.Request(url, headers=headers)
html = urlopen(req)
bs0bj = BeautifulSoup(html)
for link in bs0bj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

如果你运行代码，就会看到维基百科上凯文·贝肯词条里所有指向其他词条的链接。

当然，写程序来找出这个静态的维基百科词条里所有的词条链接很有趣，不过没什么实际用处。我们需要让这段程序更像下面的形式。

一个函数 getLinks，可以用维基百科词条 /wiki/< 词条名称 > 形式的 URL 链接作为参数，然后以同样的形式返回一个列表，里面包含所有的词条 URL 链接。

一个主函数，以某个起始词条为参数调用 getLinks，再从返回的 URL 列表里随机选择一个词条链接，再调用 getLinks，直到我们主动停止，或者在新的页面上没有词条链接了，程序才停止运行。

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import random
import re


_url = "https://zh.wikipedia.ahau.cf"
articalUrl = "/wiki/%E5%87%AF%E6%96%87%C2%B7%E8%B4%9D%E8%82%AF"
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}


def getlinks(_url, articalUrl):
    url = _url + articalUrl
    req = request.Request(url, headers=headers)
    try:
        html = urlopen(req)
    except HTTPError as e:
        return None
    try:
        bs0bj = BeautifulSoup(html)
        return bs0bj.find("div", {"id": "bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))
    except AttributeError as e:
        return None

if __name__ == '__main__':
    random.seed(time.time())
    links = getlinks(_url, articalUrl)
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links) - 1)].attrs['href']
        print(newArticle)
        links = getlinks(_url, newArticle)

导入需要的 Python 库之后，程序首先做的是用系统当前时间生成一个随机数生成器。这样可以保证在每次程序运行的时候，维基百科词条的选择都是一个全新的随机路径。

### 伪随机数和随机数种子
- 在前面的示例中，为了能够连续地随机遍历维基百科，我用 Python 的随机数生成器来随机选择每一页上的一个词条链接。但是，用随机数的时候需要格外小心。
- 虽然计算机很擅长做精确计算，但是它们处理随机事件时非常不靠谱。因此，随机数是一个难题。大多数随机数算法都努力创造一种呈均匀分布且难以预测的数据序列，但是在算法初始化阶段都需要提供随机数“种子”（random seed）。而完全相同的种子每次将产生同样的“随机”数序列，因此我用系统时间作为随机数序列生成的起点。这样做会让程序运行的时候更具有随机性。
- 其实，Python 的伪随机数（pseudorandom number）生成器用的是梅森旋转（Mersenne Twister）算法（https://en.wikipedia.org/wiki/Mersenne_Twister） ， 它产生的随机数很难预测且呈均匀分布，就是有点儿耗费 CPU 资源。真正好的随机数可不便宜！

# 2 - 采集整个网站

在上一节内容里，我们实现了在一个网站上随机地从一个链接跳到另一个链接。但是，如果你需要系统地把整个网站按目录分类，或者要搜索网站上的每一个页面，怎么办？那就得采集整个网站，那是一种非常耗费内存资源的过程，尤其是处理大型网站时，最合适的工具就是用一个数据库来储存采集的资源。

### 深网和暗网
- 你可能听说过深网（deep Web）、暗网（dark Web）或隐藏网络（hidden Web）之类的术语，尤其是在最近的媒体中。它们是什么意思呢？
- 深网是网络的一部分，与浅网（surface Web）对立。浅网是互联网上搜索引擎可以抓到的那部分网络。据不完全统计，互联网中其实约 90% 的网络都是深网。因为谷歌不能做像表单提交这类事情，也找不到那些没有直接链接到顶层域名上的网页，或者因
为有 robots.txt 禁止而不能查看网站，所以浅网的数量相对深网还是比较少的。
- 暗网，也被称为 Darknet 或 dark Internet，完全是另一种“怪兽”。它们也建立在已有的网络基础上，但是使用 Tor 客户端，带有运行在 HTTP 之上的新协议，提供了一个信息交换的安全隧道。这类暗网页面也是可以采集的，就像你采集其他网站一样，不过这些内容超出了本书的范围。
- 和暗网不同，深网是相对容易采集的。实际上，本书的很多工具都是在教你如何采集那些 Google 爬虫机器人不能获取的深网信息。

为了避免一个页面被采集两次，链接去重是非常重要的。在代码运行时，把已发现的所有链接都放到一起，并保存在方便查询的列表里（下文示例指 Python 的集合 set 类型）。只有“新”链接才会被采集，之后再从页面中搜索其他链接：

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import time
import random
import re

articalUrl = "/wiki/%E5%87%AF%E6%96%87%C2%B7%E8%B4%9D%E8%82%AF"
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

pages = set()
def getlinks(articalUrl):
    global pages
    url = "https://zh.wikipedia.ahau.cf" + articalUrl
    req = request.Request(url, headers=headers)
    html = urlopen(req)
    bs0bj = BeautifulSoup(html)
    for link in bs0bj.findAll("a", href=re.compile("^(/wiki/)")):
        if "href" in link.attrs:
            if link.attrs["href"] not in pages:
                newpage = link.attrs["href"]
                print(newpage)
                pages.add(newpage)
                getlinks(newpage)
getlinks("")

一开始，用 getLinks 处理一个空 URL，其实是维基百科的主页，因为在函数里空 URL 就 是 http://en.wikipedia.org 。然后，遍历首页上每个链接，并检查是否已经在全局变量集合 pages 里面了（已经采集的页面集合）。如果不在，就打印到屏幕上，并把链接加入pages 集合，再用 getLinks 递归地处理这个链接。

### 关于递归的警告
- 这个警告在软件开发书籍里很少提到，但是我觉得你应该注意：如果递归运行的次数非常多，前面的递归程序就很可能崩溃。Python 默认的递归限制（程序递归地自我调用次数）是 1000 次。因为维基百科的网络链接浩如烟海，所以这个程序达到递归限制后就会停止，除非你设置一个较大的递归计数器，或用其他手段不让它停止。
- 对于那些链接深度少于1000的“普通”网站，这个方法通常可以正常运行，一些奇怪的异常除外。例如，我曾经遇到过一个网站，有一个在生成博文内链的规则。这个规则是“当前页面把 /blog/title_of_blog.php 加到它后面，作为本页面的URL接”。 
- 问题是它们可能会把 /blog/title_of_blog.php 加到一个已经有 /blog/ 的 URL 上面了。因此，网站就多了一个 /blog/。最后，我的爬虫找到了这样的 URL 链接：/blog/blog/blog/blog.../blog/title_of_blog.php。 后来，我增加了一些条件，对可能导致无限循环的部分进行检查，确保那些URL 不是这么荒谬。但是，如果你不去检查这些问题，爬虫很快就会崩溃。

### 收集整个网站数据

当然，如果只是从一个页面跳到另一个页面，那么网络爬虫是非常无聊的。为了有效地使用它们，在用爬虫的时候我们需要在页面上做些事情。让我们看看如何创建一个爬虫来收集页面标题、正文的第一个段落，以及编辑页面的链接（如果有的话）这些信息。

和往常一样，决定如何做好这些事情的第一步就是先观察网站上的一些页面，然后拟定一个采集模式。通过观察几个维基百科页面，包括词条和非词条页面，比如隐私策略之类的页面，就会得出下面的规则。

- 所有的标题（所有页面上，不论是词条页面、编辑历史页面还是其他页面）都是在h1 → span 标签里，而且页面上只有一个 h1 标签。
- 前面提到过，所有的正文文字都在 div#bodyContent 标签里。但是，如果我们想更进一步获取第一段文字，可能用 div#mw-content-text → p 更好（只选择第一段的标签）。这个规则对所有页面都适用，除了文件页面（例如，https://en.wikipedia.org/wiki/File:Orbit_of_274301_Wikipedia.svg） ，页面不包含内容文字（content text）的部分内容。
- 编辑链接只出现在词条页面上。如果有编辑链接，都位于 li#ca\-edit 标签的 li#ca\-edit → span → a 里面。

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

articalUrl = "/wiki/%E5%87%AF%E6%96%87%C2%B7%E8%B4%9D%E8%82%AF"
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}


pages = set()
def getlinks(articalUrl):
    global pages
    url = "https://zh.wikipedia.ahau.cf" + articalUrl
    req = request.Request(url, headers=headers)
    html = urlopen(req)
    bs0bj = BeautifulSoup(html)
    try:
        print(bs0bj.find("h1").get_text())
        print(bs0bj.find(id="mw-content-text").findAll("p")[0])
        print(bs0bj.find(id="ca-edit").find("span").find("a").attrs["href"])
    except AttributeError as e:
        print("页面缺少一些属性，不过不用担心！")
    for link in bs0bj.findAll("a", href=re.compile("^(/wiki/)")):
        if "href" in link.attrs:
            if link.attrs["href"] not in pages:
                newpage = link.attrs["href"]
                print(newpage)
                pages.add(newpage)
                getlinks(newpage)
getlinks("")